<a href="https://colab.research.google.com/github/ekdnam/allennlp-guide/blob/add-notebooks-quick-start/notebooks/quick_start/quick_start_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Quick Start

Part 1 gives you a quick walk-through of main AllenNLP concepts and features. We’ll build a complete, working NLP model (text classifier) along the way.

# Introduction

## 1. What is text classification?

Text classification is one of the simplest NLP tasks, where the model, given some input text, predicts a label for the text. See the figure below for an illustration.

![text-classification.svg](https://raw.githubusercontent.com/allenai/allennlp-guide/master/static/part1/introduction/text-classification.svg)

There are a variety of applications of text classification, such as spam filtering, sentiment analysis, and topic detection. Some examples are shown in the table below.

|Application| Description | Input | Output |
|---|---|---| ---|
| Spam filtering | Detect and filter spam emails | Email | Spam / Not spam |
| Sentiment analysis | Detect the polarity of text | Tweet, review | Positive / Negative |
|Topic detection | Detect the topic of text | News article, blog post | Business / Tech / Sports |

## 2. Defining input and output

The first step for building an NLP model is to define its input and output. In AllenNLP, each training example is represented by an `Instance` object. An `Instance` consists of one or more `Fields`, where each `Field` represents one piece of data used by your model, either as an input or an output. `Fields` well get converted to tensors and fed to your model. The [Reading Data Chapter](https://guide.allennlp.org/reading-data) provides more details on using `Instances` and `Fields` to represent textual data.

For text classification, the input and the output are very simple. The model takes a `TextField` that represents the input text and predicts its label, which is represented by a `LabelField`:

```
# Input
text: TextField

# Output
label: LabelField
```

## 3. Reading data

![dataset-reader.png](https://raw.githubusercontent.com/allenai/allennlp-guide/master/static/part1/your-first-model/dataset-reader.svg)

The first step for building an NLP application is to read the dataset and represent it with some internal data structure.

AllenNLP uses `DatasetReaders` to read the data, whose job is to transform raw data files into `Instances` that match the input / ouput spec. Our spec for text classification is:

```
# Inputs
text: TextField

# Outputs
label: LabelField
```

We'll want one `Field` for the input and another for the output, and our model will use the inputs to predict the outputs.

We assume the dataset has a simple data file format: 
```
[text] [TAB] [label]
```

for example:

```
I like this movie a lot! [TAB] positive
This was a monstrous waste of time [TAB] negative
AllenNLP is amazing [TAB] positive
Why does this have to be so complicated? [TAB] negative
This sentence expresses no sentiment [TAB] neutral
```

# Let's begin to code

# Imports

At first, we will import the required libraries.

In [1]:
import tempfile
from typing import Dict, Iterable, List, Tuple

In [2]:
!pip install allennlp

     |████████████████████████████████| 593kB 19.0MB/s 
     |████████████████████████████████| 266kB 54.1MB/s 
     |████████████████████████████████| 133kB 24.8MB/s 
     |████████████████████████████████| 1.9MB 56.2MB/s 
     |████████████████████████████████| 317kB 52.2MB/s 
     |████████████████████████████████| 1.2MB 42.3MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 7.3MB 36.7MB/s 
     |████████████████████████████████| 3.2MB 45.0MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp37-cp37m-linux_x86_64.whl size=3388751 sha256=6073d1c844e65d56543bd252922dd1baab05914c73ada0274b1cd8c3cef3c1e3
  Stored in directory: /root/.cache/pip/wheels/26/7a/37/7dbcc30a6b4efd17b91ad1f0128b7bbf84813bd4e1cfb8c1e3
  Created wheel for overrides: filename=overrides-3.1.0-cp37-none-any.whl size=10174 sha256=6ef02ec8f7b45262da13d506c59a9242680e17855b876b6d04db1084dddeee65
  Stored in

In [3]:
import allennlp
import torch
from allennlp.data import (
    DataLoader,
    DatasetReader,
    Instance,
    Vocabulary,
    TextFieldTensors,
)
from allennlp.data.data_loaders import SimpleDataLoader
from allennlp.data.fields import LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WhitespaceTokenizer
from allennlp.models import Model
from allennlp.modules import TextFieldEmbedder, Seq2VecEncoder
from allennlp.modules.seq2vec_encoders import BagOfEmbeddingsEncoder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.nn import util
from allennlp.training.trainer import GradientDescentTrainer, Trainer
from allennlp.training.optimizers import AdamOptimizer
from allennlp.training.metrics import CategoricalAccuracy

# Making a DatasetReader

You can implement your own `DatasetReader` by inheriting from the `DatasetReader` class. At minimum, you need to override the `_read()` method, which reads the input and yields `Instances`

In [ ]:
class ClassificationTsvReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer = None,
        token_indexers: Dict[str, TokenIndexer] = None,
        max_tokens: int = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.tokenizer = tokenizer or WhitespaceTokenizer()
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_tokens = max_tokens

    def _read(self, file_path: str) -> Iterable[Instance]:
        with open(file_path, "r") as lines:
            for line in lines:
                text, sentiment = line.strip().split("\t")
                tokens = self.tokenizer.tokenize(text)
                if self.max_tokens:
                    tokens = tokens[: self.max_tokens]
                text_field = TextField(tokens, self.token_indexers)
                label_field = LabelField(sentiment)
                yield Instance({"text": text_field, "label": label_field})


This is a minimal `DatasetReader` that will return a list of classification `Instances` when you call `reader.read(file)`. This reader will take each line in the input file, split the `text` into words using a tokenizer (the `SpaCyTokenizer` shown here relies on [spaCy](https://spacy.io/)), and represent those words as tensors using a word id in a vocabulary we construct for you.

Pay special attention to the `text` and `label` keys that are used in the fields dictionary passed to the `Instance` - these keys will be used as parameter names when passing tensors into your `Model` later.

Ideally, the output label would be optional when we create the `Instances`, so that we can use the same code to make predictions on unlabeled data (say, in a demo), but for the rest of this chapter we’ll keep things simple and ignore that.

There are lots of places where this could be made better for a more flexible and fully-featured reader; see the section on [DatasetReaders](https://guide.allennlp.org/reading-data#2) for a deeper dive.

# Model 

TODO

In [ ]:
class SimpleClassifier(Model):
    def __init__(
        self, vocab: Vocabulary, embedder: TextFieldEmbedder, encoder: Seq2VecEncoder
    ):
        super().__init__(vocab)
        self.embedder = embedder
        self.encoder = encoder
        num_labels = vocab.get_vocab_size("labels")
        self.classifier = torch.nn.Linear(encoder.get_output_dim(), num_labels)

    def forward(
        self, text: TextFieldTensors, label: torch.Tensor
    ) -> Dict[str, torch.Tensor]:
        print("In model.forward(); printing here just because binder is so slow")
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)
        # Shape: (batch_size, num_tokens)
        mask = util.get_text_field_mask(text)
        # Shape: (batch_size, encoding_dim)
        encoded_text = self.encoder(embedded_text, mask)
        # Shape: (batch_size, num_labels)
        logits = self.classifier(encoded_text)
        # Shape: (batch_size, num_labels)
        probs = torch.nn.functional.softmax(logits, dim=-1)
        # Shape: (1,)
        loss = torch.nn.functional.cross_entropy(logits, label)
        return {"loss": loss, "probs": probs}


# TODO

In [ ]:
def build_dataset_reader() -> DatasetReader:
    return ClassificationTsvReader()

# TODO

In [ ]:
!wget "https://raw.githubusercontent.com/allenai/allennlp-guide/master/quick_start/data/movie_review/train.tsv"
!wget "https://raw.githubusercontent.com/allenai/allennlp-guide/master/quick_start/data/movie_review/dev.tsv"
!wget "https://raw.githubusercontent.com/allenai/allennlp-guide/master/quick_start/data/movie_review/test.tsv"

--2021-02-27 13:40:37--  https://raw.githubusercontent.com/allenai/allennlp-guide/master/quick_start/data/movie_review/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6175540 (5.9M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   5.89M  24.6MB/s    in 0.2s    

2021-02-27 13:40:38 (24.6 MB/s) - ‘train.tsv’ saved [6175540/6175540]

--2021-02-27 13:40:38--  https://raw.githubusercontent.com/allenai/allennlp-guide/master/quick_start/data/movie_review/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting respons

# TODO

In [ ]:
def read_data(reader: DatasetReader) -> Tuple[List[Instance], List[Instance]]:
    print("Reading data")
    training_data = list(reader.read("train.tsv"))
    validation_data = list(reader.read("dev.tsv"))
    return training_data, validation_data

In [ ]:
def build_vocab(instances: Iterable[Instance]) -> Vocabulary:
    print("Building the vocabulary")
    return Vocabulary.from_instances(instances)

In [ ]:
def build_model(vocab: Vocabulary) -> Model:
    print("Building the model")
    vocab_size = vocab.get_vocab_size("tokens")
    embedder = BasicTextFieldEmbedder(
        {"tokens": Embedding(embedding_dim=10, num_embeddings=vocab_size)}
    )
    encoder = BagOfEmbeddingsEncoder(embedding_dim=10)
    return SimpleClassifier(vocab, embedder, encoder)

# TODO

In [ ]:
def run_training_loop():
    dataset_reader = build_dataset_reader()

    train_data, dev_data = read_data(dataset_reader)

    vocab = build_vocab(train_data + dev_data)
    model = build_model(vocab)

    train_loader, dev_loader = build_data_loaders(train_data, dev_data)
    train_loader.index_with(vocab)
    dev_loader.index_with(vocab)

    # You obviously won't want to create a temporary file for your training
    # results, but for execution in binder for this guide, we need to do this.
    with tempfile.TemporaryDirectory() as serialization_dir:
        trainer = build_trainer(model, serialization_dir, train_loader, dev_loader)
        print("Starting training")
        trainer.train()
        print("Finished training")

    return model, dataset_reader

In [ ]:
def build_data_loaders(
    train_data: List[Instance],
    dev_data: List[Instance],
) -> Tuple[DataLoader, DataLoader]:
    train_loader = SimpleDataLoader(train_data, 8, shuffle=True)
    dev_loader = SimpleDataLoader(dev_data, 8, shuffle=False)
    return train_loader, dev_loader

In [ ]:
def build_trainer(
    model: Model,
    serialization_dir: str,
    train_loader: DataLoader,
    dev_loader: DataLoader,
) -> Trainer:
    parameters = [(n, p) for n, p in model.named_parameters() if p.requires_grad]
    optimizer = AdamOptimizer(parameters)  # type: ignore
    trainer = GradientDescentTrainer(
        model=model,
        serialization_dir=serialization_dir,
        data_loader=train_loader,
        validation_data_loader=dev_loader,
        num_epochs=5,
        optimizer=optimizer,
    )
    return trainer

# TODO

In [ ]:
run_training_loop()

Reading data


building vocab:  12%|#1        | 208/1800 [00:00<00:00, 2073.55it/s]

Building the vocabulary


building vocab: 100%|##########| 1800/1800 [00:01<00:00, 1702.41it/s]


Building the model


...

batch_loss: 0.0028, loss: 0.0020 ||:  96%|#########6| 193/200 [00:05<00:00, 29.82it/s]

In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow


batch_loss: 0.0025, loss: 0.0019 ||: 100%|##########| 200/200 [00:05<00:00, 33.38it/s]
batch_loss: 0.1532, loss: 0.3453 ||: 100%|##########| 25/25 [00:00<00:00, 420.96it/s]


In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); printing here just because binder is so slow
In model.forward(); print

(SimpleClassifier(
   (embedder): BasicTextFieldEmbedder(
     (token_embedder_tokens): Embedding()
   )
   (encoder): BagOfEmbeddingsEncoder()
   (classifier): Linear(in_features=10, out_features=2, bias=True)
 ), <__main__.ClassificationTsvReader at 0x7f914f2cd150>)